# **CA13**: Ejercicio 2 || MEDIANTE HILOS

Los enlaces se visualizarán de la siguiente manera: <span style="color: lightblue; text-decoration:underline;">**ENLACE**</span>.</br>
Los comandos / datos importantes a notar se visualizarán de la siguiente manera: <span style="color: green;">**COMANDO**</span>.

## INTRODUCCIÓN 🎄

Para entender el concepto de **hilos**, se recomienda visualizar [<span style="color: lightblue; text-decoration:underline;">__EL SIGUIENTE VÍDEO.__</span>](https://www.youtube.com/watch?v=JMoM9AliF5E). </br>

En este ejercicio implementaremos un programa que utiliza **hilos** para realizar tareas concurrentes, integrando memoria compartida y serialización de datos en un archivo. Para ello, es importante tener claros los siguientes conceptos:

---

### HILOS EN PROGRAMACIÓN 👥
Un hilo es una unidad de ejecución dentro de un proceso que comparte el mismo espacio de memoria con otros hilos del mismo proceso. Esto permite que los hilos cooperen fácilmente y accedan a los mismos datos sin necesidad de comunicación explícita, como pipes.

- **Creación de hilos**: Los hilos se crean utilizando bibliotecas como `pthread.h` (en C) o `std::thread` (en C++).
- **Comunicación**: No se necesitan pipes, ya que los hilos comparten la memoria del proceso.
- **Sincronización**: Es fundamental usar mecanismos como **<span style="color: green;">mutexes</span>** o **<span style="color: green;">barriers</span>** para evitar condiciones de carrera al trabajar con datos compartidos.

---

### IMPLEMENTACIÓN DE MEMORIA COMPARTIDA 🔁
La **memoria compartida** permite que los hilos accedan a una misma región de memoria, facilitando la colaboración. En este ejercicio:
1. Se crea un bloque de memoria compartida que contendrá los arrays generados por cada hilo.
2. Cada hilo accede a una sección de esta memoria para almacenar su array aleatorio.
3. El hilo principal (padre) accede a esta memoria para consolidar y serializar los resultados en un archivo.

---

### GENERACIÓN DE ARRAYS ALEATORIOS 🗃️
Un array aleatorio es una estructura de datos cuyos elementos tienen valores generados de manera aleatoria. En este ejercicio:
- Cada hilo genera un array de enteros aleatorios de un tamaño específico, asignado por el padre.
- **Semilla única por hilo**: Para evitar duplicados, la semilla se establece con `time(NULL)` y el identificador del hilo.

---

### MANEJO DE FICHEROS 📁
El hilo principal utiliza manejo de ficheros para escribir los resultados finales en un archivo de texto. Cada array generado es serializado y almacenado en el archivo `arrays.txt`, permitiendo revisar los datos generados de manera ordenada.

---

### ESTRUCTURA DEL PROYECTO 🛠️
El flujo general del programa será el siguiente:

1. **Creación de memoria compartida**: Se reserva un bloque de memoria para almacenar los arrays generados.
2. **Creación de hilos**: Se generan N hilos, cada uno encargado de generar un array aleatorio.
3. **Generación de arrays aleatorios**: Cada hilo genera un array y lo almacena en la memoria compartida.
4. **Serialización de resultados**: El hilo principal escribe los datos generados en un archivo de texto.
5. **Liberación de recursos**: Se libera la memoria compartida y se asegura la correcta finalización del programa.

---

### LIBRERÍAS NECESARIAS
1. **`<pthread.h>`**:
   - Proporciona funciones para la creación y manejo de hilos.
   - Ejemplo: `pthread_create()` y `pthread_join()`.
</p>

2. **`<cstdlib>`**:
   - Ofrece funciones para la generación de números aleatorios y control del programa.
   - Ejemplo: `rand()`, `srand()`, `exit()`.
</p>

3. **`<ctime>`**:
   - Permite obtener el tiempo actual como semilla para el generador de números aleatorios.
   - Ejemplo: `time(NULL)`.
</p>

4. **`<fstream>`**:
   - Proporciona funcionalidad para manejo de archivos en C++.
   - Ejemplo: `std::ofstream` para abrir y escribir en un archivo.
</p>

5. **`<iostream>`**:
   - Para imprimir mensajes en la consola durante la ejecución del programa.
   - Ejemplo: `std::cout`.
</p>

6. **`<vector>`**:
   - Permite manejar estructuras dinámicas para almacenar datos temporalmente.
   - Ejemplo: `std::vector`.
</p>

7. **`<cstring>`**:
   - Contiene funciones para manipular cadenas de caracteres y bloques de memoria.
   - Ejemplo: `memcpy()`.

---

### FUNCIONES PRINCIPALES DEL PROGRAMA

#### **1. Creación y manejo de hilos:**
   - `pthread_create()`: Crea un hilo para ejecutar una función específica.
   - `pthread_join()`: Sincroniza el hilo principal con los hilos secundarios para esperar su finalización.

#### **2. Generación de números aleatorios:**
   - `rand()` y `srand()`: Generan números aleatorios basados en una semilla.
   - `time(NULL)`: Proporciona una semilla basada en la hora actual.

#### **3. Manejo de memoria compartida:**
   - Los hilos comparten la memoria del proceso automáticamente, por lo que no se necesitan funciones adicionales para asignación como `mmap()`.

#### **4. Manejo de ficheros:**
   - `std::ofstream`: Permite abrir un archivo y escribir en él los arrays generados.

---


In [1]:
#include <sys/types.h>
#include <unistd.h>
#include <sys/wait.h>
#include <sys/mman.h> 
#include <fcntl.h>
#include <cstring>
#include <vector>
#include <fstream> 
#include <cstdlib> 
#include <ctime> 

//Las bibliotecas nos permitirán trabajar con funciones de sistema que 
//son necesarias para crear y gestionar procesos, así como para utilizar 
//la memoria compartida y realizar operaciones de entrada y salida.

//Definición de la memoria compartida que se utilizará como identificador cuando se cree y
//acceda a esta.
#define SHM_NAME "/shared_mem"
#define N 5 // Número de hilos

struct ThreadData {
    int size;         // Tamaño del array
    int *shared_mem;  // Puntero a la memoria compartida
    int offset;       // Offset en la memoria compartida
};

// Función que genera un array de enteros aleatorios.
void generateRandomArray(int *array, int size);

// Función ejecutada por cada hilo
void *threadFuncion(void *arg);

In [2]:
// Función que genera un array aleatorio
void generateRandomArray(int *array, int size) {
    srand(time(NULL) + pthread_self());  // Semilla única por hilo
    for (int i = 0; i < size; ++i) {
        array[i] = rand() % 100;  // Números entre 0 y 99
    }
}

In [3]:
// Función ejecutada por cada hilo
void *threadFunction(void *arg) {
    ThreadData *data = (ThreadData *)arg;

    // Generar array aleatorio
    int *array = data->shared_mem + data->offset;
    generateRandomArray(array, data->size);

    pthread_exit(nullptr);
}


In [4]:
int main() {
    int sizes[N] = {10, 20, 15, 30, 25};  // Tamaños de los arrays
    int totalSize = 0;                    // Tamaño total de la memoria compartida

    // Calcular tamaño total
    for (int i = 0; i < N; ++i) {
        totalSize += sizes[i];
    }

    // Crear memoria compartida
    int *shared_mem = (int *)malloc(totalSize * sizeof(int));
    if (shared_mem == nullptr) {
        perror("Error creando la memoria compartida");
        return 1;
    }

    pthread_t threads[N];          // IDs de los hilos
    ThreadData threadData[N];      // Datos para cada hilo
    int offset = 0;

    // Crear hilos
    for (int i = 0; i < N; ++i) {
        threadData[i].size = sizes[i];
        threadData[i].shared_mem = shared_mem;
        threadData[i].offset = offset;
        offset += sizes[i];

        if (pthread_create(&threads[i], nullptr, threadFunction, &threadData[i]) != 0) {
            perror("Error creando hilo");
            free(shared_mem);
            return 1;
        }
    }

    // Esperar a que los hilos terminen
    for (int i = 0; i < N; ++i) {
        pthread_join(threads[i], nullptr);
    }

    // Serializar resultados en un archivo
    std::ofstream outfile("arrays_e2.txt");
    if (outfile.is_open()) {
        offset = 0;
        for (int i = 0; i < N; ++i) {
            outfile << "Array " << i + 1 << ": ";
            for (int j = 0; j < sizes[i]; ++j) {
                outfile << shared_mem[offset + j] << " ";
            }
            outfile << "\n";
            offset += sizes[i];
        }
        outfile.close();
    } else {
        perror("Error abriendo el archivo para escritura");
    }

    // Liberar memoria compartida
    free(shared_mem);

    return 0;
}

main();

# EJERCICIO PROPUESTO

Implemente un programa en C++ que cumpla con las siguientes especificaciones:

1. El programa debe utilizar procesos hijos para generar arrays aleatorios de enteros.
2. Cada proceso hijo calculará la suma de los elementos del array que generó.
3. La suma será enviada al proceso padre utilizando pipes como medio de comunicación.
4. El proceso padre recibirá las sumas y las imprimirá en la terminal junto con los arrays generados

***Por lo tanto ya no utilizaremos HILOS sino procesos***.

En vez de generar un archivo nuevo, tendrás que **imprimir el resultado por pantalla**.

Además, se recomienda **reducir** el tamaño de el número de procesos hijos de *5 a 3*.

- Modifica el código que hemos hecho anteriormente y utilizalo como esqueleto.